In [1]:
from scraper import WebScraper
from time import sleep
import pandas as pd
from json import dumps
from tqdm import tqdm
from bs4 import BeautifulSoup
from glob import glob

In [2]:
cp = pd.read_excel('./cp_catalogue.xlsx', dtype=str)
cp.drop_duplicates(subset='Código Postal', inplace=True)
cp['Código Postal'] = cp['Código Postal'].str.zfill(5)
already_scraped = glob('./results/rappi/*')
already_scraped = set(a.split('/')[-1].split('.')[0].split('_')[-1] for a in already_scraped)
cp = cp[~cp['Código Postal'].isin(already_scraped)]

direcciones = [f'{c}, Ciudad de México' for c in cp['Código Postal']]

In [3]:
source_cates = dict(
    populares_cerca = "https://www.rappi.com.mx/"
    )

source_cate = "populares_cerca"

In [4]:
for current_address in tqdm(direcciones):
    
    scraper = WebScraper()

    scraper.navigate_to_url(source_cates["populares_cerca"])

    scraper.click_element('//*[@id="rappi-web-toolbar"]/div[2]/div/div/div/span')

    final_address = scraper.enter_address(current_address,
                                        input_xpath="//div[starts-with(@id, 'chakra-modal--body-')]//input",
                                        selection_xpath="//div[starts-with(@id, 'chakra-modal--body-')]//ul/li[1]/button"
                                        )

    # Confirm address
    try:
        scraper.click_element('//*[@id="confirm-address-button"]')

        sleep(2)
        # Save address
        scraper.click_element('//*[@id="save-address-button"]')
    except:
        scraper.close()
        
        with open(f'./results/rappi/error_{current_address.split(',')[0]}.txt', 'w') as f:
            f.write('Error al seleccionar dirección (No data)\n')
            
        continue

    cards_data = scraper.scrape_cards(
        cards_xpath='//*[@id="__next"]/div[3]/div[2]/div/div[2]/div',
        aditional_info={"cate_source": source_cate,
                        "current_address": current_address,
                        "final_address": final_address,
                        "platform": 'rappi'
                        }
    )
    
    
    with open(f'./results/rappi/rappi_{source_cate}_{current_address.split(',')[0]}.json', 'w') as f:
        f.write(dumps(cards_data, indent=4))
    
    soup = BeautifulSoup(scraper.driver.page_source, 'html.parser')
    with open(f'./results/rappi/rappi_{source_cate}_{current_address.split(',')[0]}.html', 'w') as f:
        f.write(soup.prettify()) 
    
    
    scraper.close()
    
    sleep(1)

  0%|          | 0/1110 [00:00<?, ?it/s]

2025-02-02 07:47:41,789 - INFO - Navegando a la URL: https://www.rappi.com.mx/
2025-02-02 07:47:48,092 - INFO - Elemento en //*[@id="rappi-web-toolbar"]/div[2]/div/div/div/span clickeado correctamente.
2025-02-02 07:47:52,731 - INFO - Elemento en //div[starts-with(@id, 'chakra-modal--body-')]//ul/li[1]/button clickeado correctamente.
2025-02-02 07:47:52,732 - INFO - Dirección ingresada correctamente.
2025-02-02 07:48:02,913 - ERROR - No se pudo hacer clic en el elemento en //*[@id="confirm-address-button"]: Message: 
Stacktrace:
#0 0x61390fe4eb9a <unknown>
#1 0x61390fb34670 <unknown>
#2 0x61390fb83c48 <unknown>
#3 0x61390fb83ee1 <unknown>
#4 0x61390fbca924 <unknown>
#5 0x61390fba8bad <unknown>
#6 0x61390fbc7dd6 <unknown>
#7 0x61390fba8923 <unknown>
#8 0x61390fb766e7 <unknown>
#9 0x61390fb776de <unknown>
#10 0x61390fe1866b <unknown>
#11 0x61390fe1c611 <unknown>
#12 0x61390fe044e5 <unknown>
#13 0x61390fe1d192 <unknown>
#14 0x61390fde96ef <unknown>
#15 0x61390fe3d9d8 <unknown>
#16 0x61390

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x6113a3002b9a <unknown>
#1 0x6113a2ce8670 <unknown>
#2 0x6113a2ccf248 <unknown>
#3 0x6113a2ccf123 <unknown>
#4 0x6113a2cea9a2 <unknown>
#5 0x6113a2d7b6af <unknown>
#6 0x6113a2d5c923 <unknown>
#7 0x6113a2d2a6e7 <unknown>
#8 0x6113a2d2b6de <unknown>
#9 0x6113a2fcc66b <unknown>
#10 0x6113a2fd0611 <unknown>
#11 0x6113a2fb84e5 <unknown>
#12 0x6113a2fd1192 <unknown>
#13 0x6113a2f9d6ef <unknown>
#14 0x6113a2ff19d8 <unknown>
#15 0x6113a2ff1ba7 <unknown>
#16 0x6113a30019ec <unknown>
#17 0x798c1269ca94 <unknown>
#18 0x798c12729c3c <unknown>
